In [1]:
!pip install numpy==1.21.6

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install CityLearn==2.1b9

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install gym==0.21.0


Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip install Shimmy

Defaulting to user installation because normal site-packages is not writeable


In [5]:
!pip install pickle5

Defaulting to user installation because normal site-packages is not writeable


In [6]:
#!pip install agents

In [7]:
#!pip install rewards

In [8]:
!pip install stable_baselines3

Defaulting to user installation because normal site-packages is not writeable


In [9]:
!pip install tqdm

Defaulting to user installation because normal site-packages is not writeable


# relancer le kernel

In [13]:
import numpy as np
import time
import os
import csv
from citylearn.citylearn import CityLearnEnv
from citylearn.reward_function import SolarPenaltyAndComfortReward, SolarPenaltyReward, ComfortReward

#from agents.user_agent import SubmissionAgent
#from rewards.user_reward import SubmissionReward
from stable_baselines3.sac import SAC
from stable_baselines3.ppo import PPO
from citylearn.wrappers import NormalizedObservationWrapper, StableBaselines3Wrapper

from tqdm import tqdm
import matplotlib.pyplot as plt 

C:\Users\gacece\AppData\Roaming\Python\Python37\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
class WrapperEnv:
    """
    Env to wrap provide Citylearn Env data without providing full env
    Preventing attribute access outside of the available functions
    """
    def __init__(self, env_data):
        self.observation_names = env_data['observation_names']
        self.action_names = env_data['action_names']
        self.observation_space = env_data['observation_space']
        self.action_space = env_data['action_space']
        self.time_steps = env_data['time_steps']
        self.seconds_per_time_step = env_data['seconds_per_time_step']
        self.random_seed = env_data['random_seed']
        self.buildings_metadata = env_data['buildings_metadata']
        self.episode_tracker = env_data['episode_tracker']
    
    def get_metadata(self):
        return {'buildings': self.buildings_metadata}

def create_citylearn_env(config, reward_function):
    env = CityLearnEnv(config.SCHEMA, reward_function=reward_function)

    env_data = dict(
        observation_names = env.observation_names,
        action_names = env.action_names,
        observation_space = env.observation_space,
        action_space = env.action_space,
        time_steps = env.time_steps,
        random_seed = None,
        episode_tracker = None,
        seconds_per_time_step = None,
        buildings_metadata = env.get_metadata()['buildings']
    )

    wrapper_env = WrapperEnv(env_data)
    return env, wrapper_env


def update_power_outage_random_seed(env: CityLearnEnv, random_seed: int) -> CityLearnEnv:
    """Update random seed used in generating power outage signals.
    
    Used to optionally update random seed for stochastic power outage model in all buildings.
    Random seeds should be updated before calling :py:meth:`citylearn.citylearn.CityLearnEnv.reset`.
    """

    for b in env.buildings:
        b.stochastic_power_outage_model.random_seed = random_seed

    return env

In [18]:
class Config:
    data_dir = './data/'
    SCHEMA = os.path.join(data_dir, 'schemas/warm_up/schema.json')
    num_episodes = 1
    
config = Config()

In [19]:
env, wrapper_env = create_citylearn_env(config, ComfortReward)

In [20]:
env = NormalizedObservationWrapper(env)
env = StableBaselines3Wrapper(env)

# Test Trainning

In [21]:
agent = PPO('MlpPolicy', env)
agent.learn(total_timesteps=env.time_steps*1)

C:\Users\gacece\AppData\Roaming\Python\Python37\site-packages\stable_baselines3\common\vec_env\patch_gym.py:50: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  "You provided an OpenAI Gym environment. "


In [ ]:
agent.save("models/SubmissionRewardPPO/SacModel_1epoch")

In [ ]:
agent = PPO('MlpPolicy', env)
agent.load("models/SubmissionRewardPPO/SacModel_1epoch")

# Real Trainning

In [ ]:
agent = PPO('MlpPolicy', env)

In [ ]:
#agent.load("models/ComfortReward/SacModel_38epoch")

for epochTrains in tqdm(range(40,80)):#changer le début du range si reprise d'entraienement
    agent.learn(total_timesteps=env.time_steps*2)
    agent.save("models/ComfortReward/SacModel_{}epoch".format(epochTrains*2))

In [ ]:
PathSave = "models/ComfortReward/"
trajectories_Full = []
episode_metrics = []
ind = 0

for model in tqdm(os.listdir(PathSave)):
    agent = PPO('MlpPolicy', env)
    if(model[:3]  == "Sac") :
        agent.load(PathSave+model[:-4])
        trajectories_Full.append([])
        observations = env.reset()
        agent_time_elapsed = 0

        step_start = time.perf_counter()
        actions = agent.predict(observations)
        agent_time_elapsed += time.perf_counter() - step_start

        episodes_completed = 0
        num_steps = 0
        interrupted = False

        while True:
            observations, rewards, done, _ = env.step(actions[0])
            if not done:
                step_start = time.perf_counter()
                actions = agent.predict(observations)

                agent_time_elapsed += time.perf_counter() - step_start
                trajectories_Full[ind].append([observations.tolist(), actions[0], rewards])
            else:
                episodes_completed += 1
                metrics_df = env.evaluate_citylearn_challenge()
                episode_metrics.append(metrics_df)
                #print(
                    #f"Episode complete: {episodes_completed} | Latest episode metrics: {metrics_df}"
                #)

                # Optional: Uncomment line below to update power outage random seed
                # from what was initially defined in schema
                env = update_power_outage_random_seed(env, 90000)

                observations = env.reset()

                step_start = time.perf_counter()
                actions = agent.predict(observations)
                agent_time_elapsed += time.perf_counter() - step_start

            num_steps += 1

            if episodes_completed >= 1:
                break

        ind += 1

In [ ]:
for i [0,100]:
    s = env.reset()
    a = agent.predict(s)
    MS,N,don = env.step(a)

# trajectoire 

In [ ]:
rewardMean = []
ind = 0
for trajectoire in trajectories_Full:
    rewardMean.append(0)
    for step in trajectoire:
        rewardMean[ind] += step[2]
    rewardMean[ind] /= len(trajectoire)
    ind+=1

In [ ]:
plt.clf()
plt.plot([k for k in range(len(trajectories_Full))], rewardMean, color="r")
plt.savefig(PathSave+"RewardMeanPerEpisode")

In [ ]:
name = "trajectories_train_1_eval_1_reward_test3"

with open("trajectories/" + name + ".csv", "w", newline='') as fichier:
    writer = csv.writer(fichier)
    writer.writerow(["observations", "actions", "rewards"])
    for i in range(len(trajectories)):
        writer.writerow(trajectories[i])

# Custom Reward Function





# KPI






In [26]:
model = PPO('MlpPolicy',env)
model.learn(total_timesteps=env.time_steps*2)

In [27]:
observations = env.reset()

In [29]:
while not env.done:
    actions, _ = model.predict(observations, deterministic=True)
    observations, _, _, _ = env.step(actions)

In [63]:
kpis = env.evaluate()
kpis = kpis.pivot(index='cost_function',columns='name',values='value')
display(kpis)

name,Building_1,Building_2,Building_3,District
cost_function,,,,
annual_normalized_unserved_energy_total,0.015386,0.011345,0.013734,0.013488
annual_peak_average,NaN,NaN,NaN,0.833569
carbon_emissions_total,0.388424,0.349106,0.483511,0.407014
cost_total,0.367213,0.324821,0.453112,0.381715
daily_one_minus_load_factor_average,NaN,NaN,NaN,1.311379
daily_peak_average,NaN,NaN,NaN,0.682663
discomfort_delta_average,9.554476,7.219607,8.290310,8.354798
discomfort_delta_maximum,16.838272,14.897806,14.197481,15.311186
discomfort_delta_minimum,-0.123569,-0.581318,-0.372210,-0.359032


In [ ]:
kpis2 = kpis2.pivot(index=)

In [31]:
env.time_steps

720

In [32]:
kpi_names = [ 'electricity_consumption', 'cost', 'carbon_emissions', 'average_daily_peak', 'ramping', '1 - load_factor']

In [33]:
import pandas as pd

In [64]:
def get_kpis(env: CityLearnEnv) -> pd.DataFrame:
    """Returns evaluation KPIs.

    Electricity consumption, cost and carbon emissions KPIs are provided
    at the building-level and average district-level. Average daily peak,
    ramping and (1 - load factor) KPIs are provided at the district level.

    Parameters
    ----------
    env: CityLearnEnv
        CityLearn environment instance.

    Returns
    -------
    kpis: pd.DataFrame
        KPI table.
    """

    kpis = env.evaluate()

    # names of KPIs to retrieve from evaluate function
    kpi_names = [
        'electricity_consumption_total', 'cost_total', 'carbon_emissions_total',
        'daily_peak_average', 'ramping_average', 'daily_one_minus_load_factor_average',
        'one_minus_thermal_resilience_proportion','power_outage_normalized_unserved_energy_total',
        'annual_peak_average'
    ]
    kpis = kpis[
        (kpis['cost_function'].isin(kpi_names))
    ].dropna()

#     # round up the values to 3 decimal places for readability
#     kpis['value'] = kpis['value'].round(3)

    kpis = kpis.pivot(index='cost_function',columns='name',values='value')

    # rename the column that defines the KPIs
    kpis = kpis.rename(columns={'cost_function': 'kpi'})
    
    

    return kpis

In [65]:
kpis_pd = get_kpis(env)

In [66]:
display(kpis_pd)

name,Building_1,Building_2,Building_3,District
cost_function,,,,
annual_peak_average,NaN,NaN,NaN,0.833569
carbon_emissions_total,0.388424,0.349106,0.483511,0.407014
cost_total,0.367213,0.324821,0.453112,0.381715
daily_one_minus_load_factor_average,NaN,NaN,NaN,1.311379
daily_peak_average,NaN,NaN,NaN,0.682663
electricity_consumption_total,0.392086,0.355929,0.489196,0.412404
one_minus_thermal_resilience_proportion,1.000000,1.000000,1.000000,1.000000
power_outage_normalized_unserved_energy_total,0.579528,0.488515,0.551207,0.539750
ramping_average,NaN,NaN,NaN,0.884492


In [36]:
from typing import List, Mapping, Tuple

In [37]:
def plot_building_kpis(envs: Mapping[str, CityLearnEnv]) -> plt.Figure:
    """Plots electricity consumption, cost and carbon emissions
    at the building-level for different control agents in bar charts.

    Parameters
    ----------
    envs: Mapping[str, CityLearnEnv]
        Mapping of user-defined control agent names to environments
        the agents have been used to control.

    Returns
    -------
    fig: plt.Figure
        Figure containing plotted axes.
    """

    kpis_list = []

    for k, v in envs.items():
        kpis = get_kpis(v)
        kpis = kpis[kpis['level']=='building'].copy()
        kpis['building_id'] = kpis['name'].str.split('_', expand=True)[1]
        kpis['building_id'] = kpis['building_id'].astype(int).astype(str)
        kpis['env_id'] = k
        kpis_list.append(kpis)

    kpis = pd.concat(kpis_list, ignore_index=True, sort=False)
    kpi_names= kpis['kpi'].unique()
    column_count_limit = 3
    row_count = math.ceil(len(kpi_names)/column_count_limit)
    column_count = min(column_count_limit, len(kpi_names))
    building_count = len(kpis['name'].unique())
    env_count = len(envs)
    figsize = (3.0*column_count, 0.3*env_count*building_count*row_count)
    fig, _ = plt.subplots(
        row_count, column_count, figsize=figsize, sharey=True
    )

    for i, (ax, (k, k_data)) in enumerate(zip(fig.axes, kpis.groupby('kpi'))):
        sns.barplot(x='value', y='name', data=k_data, hue='env_id', ax=ax)
        ax.axvline(1.0, color='black', linestyle='--', label='Baseline')
        ax.set_xlabel(None)
        ax.set_ylabel(None)
        ax.set_title(k)

        if i == len(kpi_names) - 1:
            ax.legend(
                loc='upper left', bbox_to_anchor=(1.3, 1.0), framealpha=0.0
            )
        else:
            ax.legend().set_visible(False)

        for s in ['right','top']:
            ax.spines[s].set_visible(False)

        for p in ax.patches:
            ax.text(
                p.get_x() + p.get_width(),
                p.get_y() + p.get_height()/2.0,
                p.get_width(), ha='left', va='center'
            )

    plt.tight_layout()
    return fig

In [67]:
plot_building_kpis(env)

AttributeError: 'CityLearnEnv' object has no attribute 'items'

In [38]:
def plot_district_kpis(envs: Mapping[str, CityLearnEnv]) -> plt.Figure:
    """Plots electricity consumption, cost, carbon emissions,
    average daily peak, ramping and (1 - load factor) at the
    district-level for different control agents in a bar chart.

    Parameters
    ----------
    envs: Mapping[str, CityLearnEnv]
        Mapping of user-defined control agent names to environments
        the agents have been used to control.

    Returns
    -------
    fig: plt.Figure
        Figure containing plotted axes.
    """

    kpis_list = []

    for k, v in envs.items():
        kpis = get_kpis(v)
        kpis = kpis[kpis['level']=='district'].copy()
        kpis['env_id'] = k
        kpis_list.append(kpis)

    kpis = pd.concat(kpis_list, ignore_index=True, sort=False)
    row_count = 1
    column_count = 1
    env_count = len(envs)
    kpi_count = len(kpis['kpi'].unique())
    figsize = (6.0*column_count, 0.225*env_count*kpi_count*row_count)
    fig, ax = plt.subplots(row_count, column_count, figsize=figsize)
    sns.barplot(x='value', y='kpi', data=kpis, hue='env_id', ax=ax)
    ax.axvline(1.0, color='black', linestyle='--', label='Baseline')
    ax.set_xlabel(None)
    ax.set_ylabel(None)

    for s in ['right','top']:
        ax.spines[s].set_visible(False)

    for p in ax.patches:
        ax.text(
            p.get_x() + p.get_width(),
            p.get_y() + p.get_height()/2.0,
            p.get_width(), ha='left', va='center'
        )

    ax.legend(loc='upper left', bbox_to_anchor=(1.3, 1.0), framealpha=0.0)
    plt.tight_layout()

    return fig

In [68]:
plot_district_kpis(env)

AttributeError: 'CityLearnEnv' object has no attribute 'items'

In [70]:
def plot_building_load_profiles(envs: Mapping[str, CityLearnEnv]) -> plt.Figure:
    """Plots building-level net electricty consumption profile
    for different control agents.

    Parameters
    ----------
    envs: Mapping[str, CityLearnEnv]
        Mapping of user-defined control agent names to environments
        the agents have been used to control.

    Returns
    -------
    fig: plt.Figure
        Figure containing plotted axes.
    """

    building_count = len(list(envs.values())[0].buildings)
    column_count_limit = 4
    row_count = math.ceil(building_count/column_count_limit)
    column_count = min(column_count_limit, building_count)
    figsize = (4.0*column_count, 1.75*row_count)
    fig, _ = plt.subplots(row_count, column_count, figsize=figsize)

    for i, ax in enumerate(fig.axes):
        for k, v in envs.items():
            y = v.buildings[i].net_electricity_consumption
            x = range(len(y))
            ax.plot(x, y, label=k)

        y = v.buildings[i].net_electricity_consumption_without_storage
        ax.plot(x, y, label='Baseline')
        ax.set_title(v.buildings[i].name)
        ax.set_xlabel('Time step')
        ax.set_ylabel('kWh')
        ax.xaxis.set_major_locator(ticker.MultipleLocator(24))

        if i == building_count - 1:
            ax.legend(
                loc='upper left', bbox_to_anchor=(1.0, 1.0), framealpha=0.0
            )
        else:
            ax.legend().set_visible(False)


    plt.tight_layout()

    return fig

In [71]:
plot_building_load_profiles(env)

AttributeError: 'CityLearnEnv' object has no attribute 'values'